In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import re
import time
import pandas as pd
import numpy as np

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
def get_items(driver):

    #driver.get('https://www.amazon.com')

    #driver.find_element_by_css_selector('a[aria-label="Amazon Top Sellers - Shop now"]').click()
    #driver.implicitly_wait(5)

    #driver.find_element_by_xpath("//span[text()='See all results']").click()
    #driver.implicitly_wait(3)

    items_list = []


    for i in range(2, 10):
        link = f'https://www.amazon.com/s?i=specialty-aps&rh=n%3A17938598011&fs=true&page={str(i)}&qid=1619749581&ref=sr_pg_{str(i)}'
        driver.get(link)
        items = driver.find_elements_by_css_selector('span.a-size-medium.a-color-base.a-text-normal')


        for current_item in range(len(items)):
            driver.find_elements_by_css_selector('span.a-size-medium.a-color-base.a-text-normal')[current_item].click()
            driver.implicitly_wait(7)

            items_list.append(driver.current_url)
            driver.implicitly_wait(2)

            driver.get(link)
            driver.implicitly_wait(7)

    return items_list

items = list(set(get_items(driver)))
print(items)

In [ ]:
np.save('links', np.array(items))

In [ ]:
#items = np.load('links.npy')

In [ ]:
items.shape

In [ ]:
def get_reviews(driver, items_list, reviews_per_item=500, arr=None):

    # CSS selectors
    all_reviews_selector = "//a[text()='See all reviews']"
    next_page_selector = "//a[text()='Next page']"
    review_body_selector = 'span[data-hook="review-body"]>span'
    review_rating_selector = 'i[data-hook*="review-star-rating"]'

    if arr is not None:
        completed_items = int(arr.shape[0]/reviews_per_item)
        completed_reviews = int(arr.shape[0] - completed_items * reviews_per_item)
    else:
        completed_items = 0
        completed_reviews = 0

    for item in range(95, len(items_list)):
        rating_text = []

        driver.get(items_list[item])
        time.sleep(3)

        driver.find_element_by_xpath(all_reviews_selector).click()
        time.sleep(3)

        # click next page
        driver.find_element_by_xpath(next_page_selector).click()
        time.sleep(3)

        if item == completed_items and completed_reviews != 0:
            print(f'Skipped {completed_items} items')
            print(f'Skipped {completed_reviews} reviews')
            
            # click next page
            driver.find_element_by_xpath(next_page_selector).click()
            time.sleep(3)

            # change page number
            driver.get(re.sub('pageNumber=2', f'pageNumber={int(completed_reviews/10)+1}', driver.current_url))
            time.sleep(3)

        for page in range(int(completed_reviews/10), int(reviews_per_item/10)):
            soup = BeautifulSoup(driver.page_source)
            all_reviews = soup.select('[id^=customer_review]')

            for i in range(len(all_reviews)):
                try:
                    text = all_reviews[i].select_one(review_body_selector).getText()
                    star = " ".join(all_reviews[i].select_one(review_rating_selector)['class'])
                except:
                    print('Text')
                    print(all_reviews[i].select_one(review_body_selector).getText())
                    print('Stars')
                    print(all_reviews[i].select_one(review_rating_selector))
                    raise Error("Stop! Error selecting review's text or rating")

                rating = int(star[re.search(r'a-star-*', star).end()])
                rating_text.append((rating, text))

            if page == 0:
                driver.find_element_by_xpath(next_page_selector).click()
            else:
                driver.get(re.sub(f'pageNumber={page+1}', f'pageNumber={page+2}', driver.current_url))
            time.sleep(2)

            if (page+1) % 10 == 0:
                #print(f'Item {item+1}/{len(items_list)}: page {page+1}/{int(reviews_per_item/10)}')
                pass
            
            
        if completed_reviews != 0:
            completed_reviews = 0
        pd.DataFrame(rating_text, columns=['rating', 'text']).to_csv(f'./item_reviews/item_{item}.csv', index=False)

    return True
    

In [ ]:
get_reviews(driver, items, 500, None)

In [ ]:
items_arr = []
rows = 0
for i in range(66, 102):
    a = pd.read_csv(f'item_{i}.csv')
    items_arr.append(a)
    rows += a.shape[0]


In [ ]:
#pd.concat(items_arr, axis=0).to_csv('items_66_101.csv', index=False)